In [9]:
import numpy as np
from scipy.linalg import inv, eig
import cvxpy as cp



from src.data_driven import Plant, Cloud

from src.constants import (As, Bs) # system model
from src.constants import (T, KEY_RANGE, 
                       EPSILON_RANGE, INPUT_RANGE,
                         INITIAL_STATE_RANGE, MAX_DISTURBANCE)

In [10]:

def print_experiment_parameters():
    """
    Prints the system and experiment parameters.
    """
    print(f"Trajectory length for generating data: {T}")
    print(f"Input range for the system: {INPUT_RANGE}")
    print(f"Initial state range for the system: {INITIAL_STATE_RANGE}")
    print(f"Key range for random matrices F1 and G1: {KEY_RANGE}")
    print(f"Maximum for disturbance: {MAX_DISTURBANCE}")




# System and experiment parameters
print_experiment_parameters()

Trajectory length for generating data: 20
Input range for the system: 5
Initial state range for the system: 2.5
Key range for random matrices F1 and G1: 1
Maximum for disturbance: 0


In [11]:
# initializing plant and cloud 
plant = Plant(As, Bs)
cloud = Cloud()


In [12]:
n = plant.num_states # number of states
m = plant.num_inputs # number of inputs

Kbar = np.zeros((m, n)) # place holder for Kbar
epsilon_results = np.zeros((len(EPSILON_RANGE), 1)) # place holder for epsilon_result

In [13]:
"""
What we do:
1) collect data (X1, X0, U0) from a system
2) generate the keys F1 and G1 for transforming the collected data 
3) Using the matrices F1 and G1 transform the collected data 
"""


X1, X0, U0, D0 = plant.collecting_trajectories( 
                        T, INPUT_RANGE, INITIAL_STATE_RANGE, MAX_DISTURBANCE)
F1, G1 = plant.key_generation(n, m, KEY_RANGE)
X1_tilde, X0_tilde, V0 = plant.transforming_data(X1, X0, U0, F1, G1)


In [14]:
"""
What Cloud does:
1) receives the transformed data X1, X0, V0
2) solves an optimization to get the controller Kbar
and epsilonbar  
"""


bA, bB, bC, bQ = cloud.ellipsoid_parameters(X1_tilde, X0_tilde,
                                                    V0, MAX_DISTURBANCE)

for ep, epsilon in enumerate(EPSILON_RANGE):

    try:
        prob, P, Y = cloud.get_controller_cvxpy(bA, bB, bC, epsilon)
        if prob.status == "infeasible":
            epsilon_results[[ep], [0]] = -1
        else:
            epsilon_results[[ep], [0]] = epsilon
            Kbar = Y @ inv(P)
    except:
        epsilon_results[[ep], [0]] = -1 # for any other reason we get a flag



eps_bar = np.max(epsilon_results, axis=0).reshape(-1,1)
        
                                                                           

In [15]:

K = F1 + (np.eye(m) + G1) @ Kbar
eig_closed_loop = np.abs(eig(As + Bs @ K,  left=False, right=False))


print(f"""epsilonbar received from Cloud: {eps_bar}
      Controller received from Cloud: {Kbar}
      Controller for the system: {K}
      Absolute value of eigenvalues for the closed-loop system: {eig_closed_loop}
      Max eigen value: {np.max(eig_closed_loop)}
       """)


epsilonbar received from Cloud: [[0.04]]
      Controller received from Cloud: [[-0.90984865 -0.30781933 -1.3193689   0.13786905]
 [ 4.22643558  0.09468397  2.25648967 -3.01372547]]
      Controller for the system: [[ 0.94326801 -1.2143985   0.38951844 -1.77891756]
 [ 4.90371504 -0.12059297  3.29784135 -3.11565003]]
      Absolute value of eigenvalues for the closed-loop system: [0.02150225 0.24275074 0.46028111 0.51228674]
      Max eigen value: 0.5122867377255262
       


In [16]:
# save the date; we need it to simulate the bias injection attack case;
#  see the corresponding notebook
np.savez("clean_data_epsilon_bar",
          F1 = F1,
          G1 = G1,
          K_bar = Kbar,
          eps_bar = eps_bar,
          T = T,
          key_range = KEY_RANGE,
          input_range = INPUT_RANGE,
          initial_state_range = INITIAL_STATE_RANGE
          )
